- Assignment: Create a dataset containing monthly CRSP returns and book-to-market ratios for 1970 through 2023.  Use the Fama-French June 30 convention for book-to-market.  Follow the Fama-French definitions for book equity and market equity (variable definitions are at French's data library).
- We'll just do the book-to-market part.  Follow the WRDS lecture for merging with CRSP.
- Book 

In [6]:
import pandas as pd 
import numpy as np
import wrds

In [3]:
conn = wrds.Connection() 

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [27]:
 comp = conn.raw_sql(
    """
    select gvkey, datadate as date, txditc, pstkrv, pstkl, pstk, ceq, at, lt, seq
    from comp.funda
    where datadate >= '2000-01-01' and at>0 
    and indfmt='INDL' and datafmt='STD' and popsrc='D' and consol='C'
    order by gvkey, datadate
    """, 
    date_cols=['date']
)

# convert string or float to int
comp.gvkey = comp.gvkey.astype(int)
            
def bookequity(df) :
    deferredTaxes = df.txditc.fillna(0)    
    preferredStock = df[['pstkrv','pstkl','pstk']].bfill(axis=1).iloc[:, 0].fillna(0)               
    df['seq2'] = df.ceq + df.pstk               
    df['seq3'] = np.where((df['at']>=0) & (df['lt']>=0), df['at']-df['lt'], np.nan)        
    shareholdersEquity = df[['seq','seq2','seq3']].bfill(axis=1).iloc[:, 0]
    bookEquity = shareholdersEquity + deferredTaxes - preferredStock
    bookEquity = np.where(bookEquity>0, bookEquity, np.nan)
    return pd.Series(bookEquity, index=df.index)

comp['bookEquity'] = bookequity(comp)

In [28]:
comp['date'] = pd.to_datetime(
    comp.date.map(
        lambda d: str(d.year+1)+'-06-30'
    )
)
    
# if two annual reports in one calendar year (due to change of fiscal year), keep last one
comp = comp.drop_duplicates(
    subset=['gvkey', 'date'], keep='last'
) 

In [29]:
link = conn.raw_sql(
    """
    select distinct gvkey, lpermno as permno, linkdt, linkenddt
    from crsp.Ccmxpf_linktable
    where linktype in ('LU', 'LC')
    and LINKPRIM in ('P', 'C')
    """
)

# convert strings or floats to ints
link['gvkey'] = link.gvkey.astype(int)
link['permno'] = link.permno.astype(int)

# fill in missing end dates with a future date
link['linkenddt'] = pd.to_datetime(
    link.linkenddt
).fillna(pd.Timestamp('21000101'))

# merge with Compustat data and keep rows with Compustat datadate between link date and link end date
comp = comp.merge(link, on='gvkey', how='inner')
comp = comp[
    (comp.date>=comp.linkdt) & (comp.date<=comp.linkenddt)
]

In [30]:
crsp = conn.raw_sql(
    """
    select a.permno, a.permco, a.date, abs(a.prc)*a.shrout as me
    from crsp.msf a inner join crsp.msenames b
    on a.permno=b.permno and a.date between b.namedt and b.nameendt
    and b.exchcd in (1,2,3) and b.shrcd in (10,11)
    where a.date >= '2000-01-01'
    order by a.permno, a.date
    """,
date_cols=['date']
)

# change strings or floats to integers
for col in ['permno','permco']:
    crsp[col] = crsp[col].astype(int)

# define market equity as sum of market equities of all permnos associated with a permco
crsp['me'] = crsp.groupby(['date','permco']).me.transform(sum)

C:\Users\kerry\AppData\Local\Temp\ipykernel_38572\2057758595.py:18: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  crsp['me'] = crsp.groupby(['date','permco']).me.transform(sum)


In [31]:
me = crsp.groupby("permno", group_keys=True).apply(
    lambda d: d.set_index("date").resample("YE").me.last()
)

C:\Users\kerry\AppData\Local\Temp\ipykernel_38572\2629157431.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  me = crsp.groupby("permno", group_keys=True).apply(


In [32]:
me = pd.DataFrame(me).reset_index() 
me["date"] = me.date.map(
    lambda d: str(d.year+1)+'-06-30'
)
me["date"] = pd.to_datetime(me.date)


In [33]:
comp = comp.merge(me, on=["permno", "date"], how="inner")
comp["bm"] = comp.bookEquity / comp.me

In [34]:
comp.head()

,gvkey,date,txditc,pstkrv,pstkl,pstk,ceq,at,lt,seq,seq2,seq3,bookEquity,permno,linkdt,linkenddt,me,bm
0,1004,2001-06-30,56.020,0.0,0.0,0.0,339.515,740.998,401.483,339.515,339.515,339.515,395.535,54594,1972-04-24,2100-01-01,339057.00,0.001167
1,1004,2002-06-30,55.063,0.0,0.0,0.0,340.212,701.854,361.642,340.212,340.212,340.212,395.275,54594,1972-04-24,2100-01-01,242891.58,0.001627
2,1004,2003-06-30,30.601,0.0,0.0,0.0,310.235,710.199,399.964,310.235,310.235,310.235,340.836,54594,1972-04-24,2100-01-01,164001.75,0.002078
3,1004,2004-06-30,22.601,0.0,0.0,0.0,294.988,686.621,391.633,294.988,294.988,294.988,317.589,54594,1972-04-24,2100-01-01,476112.65,0.000667
4,1004,2005-06-30,17.628,0.0,0.0,0.0,301.684,709.292,407.608,301.684,301.684,301.684,319.312,54594,1972-04-24,2100-01-01,439190.52,0.000727
